# Plot data from Rigol DS1054Z scope

### Import the libraries

In [1]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [6, 8]
plt.rcParams['figure.dpi'] = 150
import numpy as np
import math
import pandas as pd
import csv
from datetime import datetime 
import glob

### Import local libraries

In [2]:
from appvib import ClSigFeatures, ClSigCompUneven

### Define plot mode. 
Interactive mode is helpful for visuallizing the program execution

In [3]:
#%matplotlib widget

### Define function to read header values

In [4]:
def get_header(str_filename_in):
    """
    
    Pulls in the header information from the scope files
    
    Parameters
    ----------
    str_filename_in : string
        full file name to read
        
    Returns
    -------
    list : header values
    
    """
    
    # Read the CSV headers
    print(str_filename_in)
    file_handle = open(str_filename_in)
    csv_reader = csv.reader(file_handle)
    lst_header = next(csv_reader)
    print(lst_header)
    file_handle.close()

    return lst_header

### Define files to read

A helpful discussion on getting .csv files into Panda:
https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

#### Define the files to be read

Choose files 

In [5]:
lst_filenames = ['test_001_000.csv', 'test_001_001.csv', 'test_001_002.csv']

Or all files with a given extension in the current director

In [6]:
#lst_filenames = []
#for file in glob.glob("*.csv"):
#    lst_filenames.append(file)

#### Read in the first data set and visualize the results

In [7]:
help(ClSigFeatures.b_read_data_as_df)

Help on function b_read_data_as_df in module appvib:

b_read_data_as_df(self, str_filename=None)
    Read the entire file in as a pandas dataframe
    
    Parameters
    ----------
    str_filename : string
        Filename, including .csv extension,  to read. If None then filename stored
        in the class is used
    
    Returns
    --------
    lst_data : list
            pandas dataframe : dataframe with all data from the file
            numpy array, datetime : vector with date and timestamps
            numpy array, double : vector with signal sampling rates
            numpy array, double : vector with delta time interval for each signal
            list, string :  list with engineering units descriptions
            list, string : machine name



Setup the storage arrays

In [8]:
dt_timestamp_base = datetime.now()
np_d_time = np.array([])
np_d_nx = np.array([])

Loop through the files

In [ ]:
for idx, str_filename in enumerate(lst_filenames):
    class_file = ClSigFeatures([1.,2.,3.], 1.)
    lst_file = class_file.b_read_data_as_df(str_filename)

    df1 = lst_file[0]
    np_dt_timestamp = lst_file[1]
    d_seconds = 0.0
    if idx == 0:
        dt_timestamp_base = np_dt_timestamp[0]
    else:
        d_seconds = (np_dt_timestamp[0] - dt_timestamp_base).total_seconds()
    np_d_fs = lst_file[2]
    lst_str_eu = lst_file[4]
    lst_str_machine_name = lst_file[5]
    
    # Validate input
    assert( len(df1.CH1) == 1200) , 'Failed to load the correct number of samples'

    # Instantiate a new class and populate it with the file contents
    class1 = ClSigFeatures(df1.CH1, np_d_fs[0], dt_timestamp=np_dt_timestamp[0], str_machine_name=lst_str_machine_name[1])
    
    # Plotting limits, then plot the signal
    plt.rcParams['figure.figsize'] = [6, 4]
    class1.ylim_tb(ylim_tb_in=[-0.5, 0.5])
    lst_plot_sig = class1.plt_sigs(b_plot_sg=True)
    lst_plot_sig.savefig('TestPlot.pdf')
    
    # Add the second signal and plot it
    plt.rcParams['figure.figsize'] = [6, 8]
    class1.idx_add_sig(df1.CH2, d_fs=np_d_fs[1], str_point_name='CH2', dt_timestamp=np_dt_timestamp[1], 
                       str_machine_name=lst_str_machine_name[2])    
    class1.ylim_tb(ylim_tb_in=[-1.0, 1.0], idx=1)
    lst_plot_sig = class1.plt_sigs(b_plot_sg=True)
    lst_plot_sig.savefig('TestPlot2.pdf')
    
    np_d_eventtimes = class1.np_d_est_triggers(np_d_sig=class1.np_d_sig, i_direction=0, d_threshold=0.125)
    lst_plt_eventtimes = class1.plt_eventtimes(idx_eventtimes=0, idx=1)
    
    # 1X extraction
    np_d_nx = np.append(np_d_nx, class1.calc_nx(np_d_sig=class1.get_np_d_sig(idx=1), np_d_eventtimes=class1.np_d_eventtimes(),
                                          b_verbose=False))
    lst_apht = class1.plt_apht()
    
    # Time series
    np_d_time = np.append(np_d_time, class1.np_d_eventtimes()+d_seconds)

In [ ]:
plt.plot(np_d_time,np.abs(np_d_nx))

In [ ]:
class_nx = ClSigCompUneven(np_d_nx, np_d_time, str_eu='volts', str_point_name='CH1', dt_timestamp=dt_timestamp_base)
class_nx.str_plot_desc = 'Baseline run'
lst_apht = class_nx.plt_apht()

In [ ]:
lst_polor = class_nx.plt_polar()